In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
############################
# generate data
############################
mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
#############################
# construct the model
#############################

tf.reset_default_graph()

filter_in_channels = 1
filter_out_channels = 32
convolution_filter_size = (5,5,filter_in_channels,filter_out_channels)
convolution_strides = (1,1,1,1)
pooling_filter_size = (1,4,4,1)
pooling_strides = (1,4,4,1)

x = tf.placeholder(dtype=tf.float32,shape=(None,784),name='x')
x_image = tf.reshape(x,(-1,28,28,1))
y = tf.placeholder(dtype=tf.float32,shape=(None,10),name='y')
conv_filter = tf.Variable(np.random.normal(size=convolution_filter_size),dtype=tf.float32,name='convolution')
conv_out = tf.nn.relu(tf.nn.conv2d(x_image,conv_filter,convolution_strides,padding='VALID'))
pool_out = tf.nn.max_pool(conv_out,pooling_filter_size,pooling_strides,padding='VALID')

fc_input = tf.reshape(pool_out,(-1,36*filter_out_channels))

h1_size = 18
w1 = tf.Variable(np.random.normal(loc=0.0,scale=0.01,size=(36*filter_out_channels,h1_size)),dtype=tf.float32,name='w1')
b1 = tf.Variable(np.zeros(h1_size),dtype=tf.float32,name='b1')
fc_layer1 = tf.nn.relu(tf.matmul(fc_input,w1) + b1)

keep_prob = tf.placeholder(tf.float32)
fc_layer1_drop = tf.nn.dropout(fc_layer1, keep_prob)

h2_size = 10
w2 = tf.Variable(np.random.normal(loc=0.0,scale=0.01,size=(h1_size,h2_size)),dtype=tf.float32,name='w2')
b2 = tf.Variable(np.zeros(h2_size),dtype=tf.float32,name='b2')
y_hat = tf.nn.softmax(tf.matmul(fc_layer1_drop,w2) + b2)

cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(y_hat),1))
grad = tf.train.AdamOptimizer(0.0001).minimize(cost)
acc =  tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y,1) , tf.argmax(y_hat,1)),tf.float32))

In [4]:
#############################
# train & test the model
#############################
epoches = 10000
batch_size = 50
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(epoches):
        train_x, train_y = mnist_data.train.next_batch(batch_size)
        c,a,_ = sess.run([cost,acc,grad],{x:train_x,y:train_y,keep_prob:0.9})
        if i % 500 == 0:
            print i,' training... cost : ', c , ' accuracy : ', a
    print 'final accuracy : ',sess.run(acc,{x: mnist_data.test.images,y: mnist_data.test.labels,keep_prob:1})

0  training... cost :  2.30411  accuracy :  0.08
500  training... cost :  0.788275  accuracy :  0.78
1000  training... cost :  0.582735  accuracy :  0.84
1500  training... cost :  0.323418  accuracy :  0.9
2000  training... cost :  0.403309  accuracy :  0.86
2500  training... cost :  0.2355  accuracy :  0.94
3000  training... cost :  0.230954  accuracy :  0.92
3500  training... cost :  0.315547  accuracy :  0.86
4000  training... cost :  0.22072  accuracy :  0.92
4500  training... cost :  0.253824  accuracy :  0.88
5000  training... cost :  0.477962  accuracy :  0.9
5500  training... cost :  0.122048  accuracy :  0.96
6000  training... cost :  0.113115  accuracy :  0.94
6500  training... cost :  0.185603  accuracy :  0.92
7000  training... cost :  0.359845  accuracy :  0.88
7500  training... cost :  0.193512  accuracy :  0.96
8000  training... cost :  0.109174  accuracy :  0.96
8500  training... cost :  0.066943  accuracy :  0.98
9000  training... cost :  0.249042  accuracy :  0.9
9500